<a href="https://colab.research.google.com/github/ameliachoi/coursera-IBM-applied-data-science-capstone/blob/master/coursera_applied_data_science_capstone_project_jinchoi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**[Coursera] Applied Data Science Capstone**

# The Battle of Neighborhoods
## Restaurants of Seoul - "Likes" Prediction Using Foursquare API and Machine Learning

---

### 1. Introduction

In [1]:
! pip install geocoder
! pip install folium

     |████████████████████████████████| 102kB 2.2MB/s 


In [2]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json
from geopy.geocoders import Nominatim

import requests
from pandas.io.json import json_normalize

import matplotlib.cm as cm
import matplotlib.colors as colors

import folium

from urllib.request import urlopen
from bs4 import BeautifulSoup

import matplotlib.pyplot as plt
import pylab as pl

from sklearn import linear_model
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import jaccard_similarity_score, confusion_matrix, classification_report, log_loss, r2_score, mean_squared_error
import itertools

import plotly
import plotly.graph_objects as go
import plotly.express as px

print('Libraries imported.')


Libraries imported.


---

### 2. Data

#### 2.1. Data Scraping and Cleaning

In [3]:
# define a function to get the geocodes that is lats and lons using geopy
address = 'Seoul, Republic of Korea'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geographical coordinate of Seoul, Republic of Korea are {}, {}.'.format(latitude, longitude))

/usr/local/lib/python3.6/dist-packages/geopy/geocoders/osm.py:143: UserWarning:

Using Nominatim with the default "geopy/1.17.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.



The geographical coordinate of Seoul, Republic of Korea are 37.5666791, 126.9782914.


In [4]:
CLIENT_ID = 'AQLM5MDGGIQWD5VD4LJS3XXSYEJJS34SPWSFWAR1THI1GZMZ' # your Foursquare ID
CLIENT_SECRET = 'EOW1J3GXQVU2JDWGOD3KG0YV3VEG0F1JQS3CAGP1V20QMBO0' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)


LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 1000 # define radius

# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    latitude, 
    longitude, 
    radius, 
    LIMIT)

print(url)

Your credentails:
CLIENT_ID: AQLM5MDGGIQWD5VD4LJS3XXSYEJJS34SPWSFWAR1THI1GZMZ
CLIENT_SECRET:EOW1J3GXQVU2JDWGOD3KG0YV3VEG0F1JQS3CAGP1V20QMBO0
https://api.foursquare.com/v2/venues/explore?&client_id=AQLM5MDGGIQWD5VD4LJS3XXSYEJJS34SPWSFWAR1THI1GZMZ&client_secret=EOW1J3GXQVU2JDWGOD3KG0YV3VEG0F1JQS3CAGP1V20QMBO0&v=20180605&ll=37.5666791,126.9782914&radius=1000&limit=100


In [7]:
# scrape the data from the generated URL
result = requests.get(url).json()
result

{'meta': {'code': 200, 'requestId': '5f13d64c669071485ae77c97'},
 'response': {'groups': [{'items': [{'reasons': {'count': 0,
       'items': [{'reasonName': 'globalInteractionReason',
         'summary': 'This spot is popular',
         'type': 'general'}]},
      'referralId': 'e-0-4ba1a9adf964a5209bc637e3-0',
      'venue': {'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/korean_',
          'suffix': '.png'},
         'id': '4bf58dd8d48988d113941735',
         'name': 'Korean Restaurant',
         'pluralName': 'Korean Restaurants',
         'primary': True,
         'shortName': 'Korean'}],
       'id': '4ba1a9adf964a5209bc637e3',
       'location': {'address': '중구 을지로1길 38',
        'cc': 'KR',
        'city': '서울특별시',
        'country': '대한민국',
        'distance': 183,
        'formattedAddress': ['중구 을지로1길 38',
         '명동',
         '서울특별시',
         '서울특별시',
         '04522',
         '대한민국'],
        'labeledLatLngs': [{'label': 'display',
  

In [8]:
# function that extracts the category of the venue
def get_category_type(row):
  try:
    categories_list = row['categories']
  except:
    categories_list = row['venue.categories']
  
  if len(categories_list) == 0:
    return None
  else:
    return categories_list[0]['name']

In [9]:
venues = result['response']['groups'][0]['items']
nearby_venues = json_normalize(venues)

# filter columns
filtered_columns = ['venue.name',
                   'venue.categories',
                   'venue.location.lat',
                   'venue.location.lng',
                   'venue.id']
nearby_venues = nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

print('{} venues were returned by FourSquare.'.format(nearby_venues.shape[0]))

100 venues were returned by FourSquare.


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: FutureWarning:

pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead



In [10]:
nearby_venues.head()

,name,categories,lat,lng,id
0,무교동북어국집,Korean Restaurant,37.567852,126.979753,4ba1a9adf964a5209bc637e3
1,Seoul Plaza (서울광장),Pedestrian Plaza,37.565475,126.977937,4baf0f0bf964a52021e93be3
2,Läderach chocolatier suisse (레더라),Chocolate Shop,37.568153,126.978265,4caea82aaef16dcbad8ba254
3,Joo Ok (주옥),Korean Restaurant,37.564705,126.977667,572c2d36498ec2eb8308ade5
4,The Plaza Hotel (더 플라자),Hotel,37.564621,126.978060,4b058782f964a520799622e3


In [11]:
nearby_venues['categories'].unique()

array(['Korean Restaurant', 'Pedestrian Plaza', 'Chocolate Shop', 'Hotel',
       'Historic Site', 'Seafood Restaurant', 'Lounge', 'Café',
       'Sushi Restaurant', 'Palace', 'Pub', 'Bookstore',
       'Chinese Restaurant', 'Buffet', 'Gukbap Restaurant', 'Plaza',
       'Bistro', 'Burger Joint', 'Dessert Shop', 'Japanese Restaurant',
       'Coffee Shop', 'Hotel Bar', 'Tourist Information Center',
       'Tea Room', 'Bagel Shop', 'Art Museum', 'Steakhouse', 'Theater',
       'Dog Run', 'Scenic Lookout', 'BBQ Joint', 'Optical Shop',
       'Bossam/Jokbal Restaurant', 'Indie Movie Theater', 'Souvenir Shop',
       'Food Court', 'Mexican Restaurant', 'Vietnamese Restaurant',
       'Performing Arts Venue', 'Noodle House', 'Italian Restaurant',
       'Bakery', 'Cosmetics Shop', 'Indian Restaurant',
       'Gym / Fitness Center'], dtype=object)

In [12]:
resturant_list = ['Korean Restaurant', 'Chocolate Shop', 
                'Seafood Restaurant', 'Café', 'Sushi Restaurant', 
                'Pub', 'Chinese Restaurant', 'Buffet', 'Gukbap Restaurant', 
                'Bistro', 'Burger Joint', 'Dessert Shop', 'Japanese Restaurant',
                'Hotel Bar', 'Coffee Shop', 'Tea Room', 'Bagel Shop', 'Steakhouse', 
                'BBQ Joint', 'Bossam/Jokbal Restaurant', 'Beer Garden', 
                'Food Court', 'Mexican Restaurant','Vietnamese Restaurant', 'Noodle House',
                'Bakery', 'Italian Restaurant', 'Indian Restaurant']

In [13]:
nearby_venues = nearby_venues[nearby_venues['categories'].isin(resturant_list)]
nearby_venues['categories'].unique()

array(['Korean Restaurant', 'Chocolate Shop', 'Seafood Restaurant',
       'Café', 'Sushi Restaurant', 'Pub', 'Chinese Restaurant', 'Buffet',
       'Gukbap Restaurant', 'Bistro', 'Burger Joint', 'Dessert Shop',
       'Japanese Restaurant', 'Coffee Shop', 'Hotel Bar', 'Tea Room',
       'Bagel Shop', 'Steakhouse', 'BBQ Joint',
       'Bossam/Jokbal Restaurant', 'Food Court', 'Mexican Restaurant',
       'Vietnamese Restaurant', 'Noodle House', 'Italian Restaurant',
       'Bakery', 'Indian Restaurant'], dtype=object)

In [14]:
# set up to pull the 'likes' from the API based on venue ID

url_list = []
like_list = []
json_list = []

for i in list(nearby_venues.id):
  venue_url = 'https://api.foursquare.com/v2/venues/{}/likes?client_id={}&client_secret={}&v={}'.format(i, CLIENT_ID, CLIENT_SECRET, VERSION)
  url_list.append(venue_url)

for link in url_list:
  result = requests.get(link).json()
  likes = result['response']['likes']['count']
  like_list.append(likes)
print(like_list)

nearby_venues['likes'] = like_list
nearby_venues.head()

[192, 57, 41, 46, 13, 114, 50, 16, 81, 74, 109, 42, 19, 71, 79, 50, 72, 96, 139, 46, 9, 41, 30, 78, 98, 22, 22, 46, 16, 137, 92, 7, 10, 12, 20, 14, 15, 23, 16, 31, 222, 12, 40, 9, 118, 116, 16, 10, 58, 142, 27, 13, 11, 47, 12, 10, 199, 15, 8, 37, 38]


,name,categories,lat,lng,id,likes
0,무교동북어국집,Korean Restaurant,37.567852,126.979753,4ba1a9adf964a5209bc637e3,192
2,Läderach chocolatier suisse (레더라),Chocolate Shop,37.568153,126.978265,4caea82aaef16dcbad8ba254,57
3,Joo Ok (주옥),Korean Restaurant,37.564705,126.977667,572c2d36498ec2eb8308ade5,41
6,철철복집,Seafood Restaurant,37.567393,126.981310,4da7cc1393a021ab13bbe742,46
8,Be-Up Coffee (비읍커피),Café,37.566975,126.979706,5ba2e09d234724002ce3f415,13


In [15]:
# rename is something more appropriate
raw_dataset = nearby_venues
raw_dataset.head()

,name,categories,lat,lng,id,likes
0,무교동북어국집,Korean Restaurant,37.567852,126.979753,4ba1a9adf964a5209bc637e3,192
2,Läderach chocolatier suisse (레더라),Chocolate Shop,37.568153,126.978265,4caea82aaef16dcbad8ba254,57
3,Joo Ok (주옥),Korean Restaurant,37.564705,126.977667,572c2d36498ec2eb8308ade5,41
6,철철복집,Seafood Restaurant,37.567393,126.981310,4da7cc1393a021ab13bbe742,46
8,Be-Up Coffee (비읍커피),Café,37.566975,126.979706,5ba2e09d234724002ce3f415,13


#### 2.2 Data Preparation

In [16]:
# grouping specific categories
raw_dataset['categories'].unique()

array(['Korean Restaurant', 'Chocolate Shop', 'Seafood Restaurant',
       'Café', 'Sushi Restaurant', 'Pub', 'Chinese Restaurant', 'Buffet',
       'Gukbap Restaurant', 'Bistro', 'Burger Joint', 'Dessert Shop',
       'Japanese Restaurant', 'Coffee Shop', 'Hotel Bar', 'Tea Room',
       'Bagel Shop', 'Steakhouse', 'BBQ Joint',
       'Bossam/Jokbal Restaurant', 'Food Court', 'Mexican Restaurant',
       'Vietnamese Restaurant', 'Noodle House', 'Italian Restaurant',
       'Bakery', 'Indian Restaurant'], dtype=object)

In [17]:
Korean = ['Korean Restaurant', 'Seafood Restaurant',
          'Gukbap Restaurant', 'Bossam/Jokbal Restaurant']

Asian = ['Sushi Restaurant', 'Chinese Restaurant',
         'Japanese Restaurant', 'Vietnamese Restaurant', 'Indian Restaurant']

Europe = ['Bistro', 'Italian Restaurant']

American = ['Burger Joint', 'Steakhouse', 'BBQ Joint', 'Mexican Restaurant']

Bar = ['Pub', 'Hotel Bar', 'Beer Garden']

Casual = ['Chocolate Shop', 'Café', 'Buffet',
          'Dessert Shop', 'Coffee Shop', 'Tea Room',
          'Bagel Shop', 'Food Court', 'Noodle House', 'Bakery']

In [18]:
def conditions(s):
  if s['categories'] in Korean:
    return 'Korean'
  elif s['categories'] in Asian:
    return 'Asian'
  elif s['categories'] in Europe:
    return 'Europe'
  elif s['categories'] in Bar:
    return 'Bar'
  elif s['categories'] in American:
    return 'American'
  else:
    return 'Casual'

raw_dataset['categories_classified'] = raw_dataset.apply(conditions, axis=1)
raw_dataset

,name,categories,lat,lng,id,likes,categories_classified
0,무교동북어국집,Korean Restaurant,37.567852,126.979753,4ba1a9adf964a5209bc637e3,192,Korean
2,Läderach chocolatier suisse (레더라),Chocolate Shop,37.568153,126.978265,4caea82aaef16dcbad8ba254,57,Casual
3,Joo Ok (주옥),Korean Restaurant,37.564705,126.977667,572c2d36498ec2eb8308ade5,41,Korean
6,철철복집,Seafood Restaurant,37.567393,126.981310,4da7cc1393a021ab13bbe742,46,Korean
8,Be-Up Coffee (비읍커피),Café,37.566975,126.979706,5ba2e09d234724002ce3f415,13,Casual
10,Sushi Cho (스시 조),Sushi Restaurant,37.564491,126.979743,4f82efddd4f208b5bd478f0e,114,Asian
14,Wolhyang (월향),Pub,37.567916,126.975665,566033b1498e93c84b700d64,50,Bar
16,열빈,Chinese Restaurant,37.567522,126.976187,4f969714e4b07e54f0d8cdb9,16,Asian
17,La Seine (라세느),Buffet,37.565960,126.980522,4c67b557b80abe9a37cad0e5,81,Casual
18,광화문국밥,Gukbap Restaurant,37.569020,126.975582,58d48a6ada54ae0db6b5a19e,74,Korean


In [19]:
pd.crosstab(index = raw_dataset['categories_classified'],
            columns = 'count')

col_0,count
categories_classified,
American,6
Asian,13
Bar,4
Casual,22
Europe,3
Korean,13


In [20]:
# First: where to bin the data

print(np.percentile(raw_dataset['likes'], 33))
print(np.percentile(raw_dataset['likes'], 66))

18.400000000000002
54.20000000000001


In [21]:
# create a function to bin for us

def rankings(s):
  if s['likes'] <= 17:
    return 3
  elif s['likes'] <= 51:
    return 2
  else:
    return 1

raw_dataset['ranking'] = raw_dataset.apply(rankings, axis=1)
raw_dataset

,name,categories,lat,lng,id,likes,categories_classified,ranking
0,무교동북어국집,Korean Restaurant,37.567852,126.979753,4ba1a9adf964a5209bc637e3,192,Korean,1
2,Läderach chocolatier suisse (레더라),Chocolate Shop,37.568153,126.978265,4caea82aaef16dcbad8ba254,57,Casual,1
3,Joo Ok (주옥),Korean Restaurant,37.564705,126.977667,572c2d36498ec2eb8308ade5,41,Korean,2
6,철철복집,Seafood Restaurant,37.567393,126.981310,4da7cc1393a021ab13bbe742,46,Korean,2
8,Be-Up Coffee (비읍커피),Café,37.566975,126.979706,5ba2e09d234724002ce3f415,13,Casual,3
10,Sushi Cho (스시 조),Sushi Restaurant,37.564491,126.979743,4f82efddd4f208b5bd478f0e,114,Asian,1
14,Wolhyang (월향),Pub,37.567916,126.975665,566033b1498e93c84b700d64,50,Bar,2
16,열빈,Chinese Restaurant,37.567522,126.976187,4f969714e4b07e54f0d8cdb9,16,Asian,3
17,La Seine (라세느),Buffet,37.565960,126.980522,4c67b557b80abe9a37cad0e5,81,Casual,1
18,광화문국밥,Gukbap Restaurant,37.569020,126.975582,58d48a6ada54ae0db6b5a19e,74,Korean,1


#### 2.3 Exploratory Data Analysis

##### The Greatest & Worst resturant in Seoul

In [22]:
# show the greatest restaurant (for these likes) list
first = raw_dataset[raw_dataset['ranking'] == 1].sort_values(by = 'likes', ascending=False)
first.head(1)

,name,categories,lat,lng,id,likes,categories_classified,ranking
66,만족오향족발,Bossam/Jokbal Restaurant,37.563408,126.975884,4ba33ce1f964a520363138e3,222,Korean,1


In [23]:
# show the worst restaurant (for these likes) list
last = raw_dataset[raw_dataset['ranking'] == 3].sort_values(by = 'likes', ascending=True)
last.head(1)

,name,categories,lat,lng,id,likes,categories_classified,ranking
52,서울고기집,BBQ Joint,37.566631,126.981539,56bbfdfb498ec350d60d9d74,7,American,3


##### The Ratios for features in the dataset

In [24]:
count_class = pd.crosstab(index = raw_dataset['categories_classified'],
            columns = 'count')

In [25]:
count_class.index

Index(['American', 'Asian', 'Bar', 'Casual', 'Europe', 'Korean'], dtype='object', name='categories_classified')

In [26]:
# the ratio of categories_classified

labels = count_class.index
values = count_class['count']

fig = go.Figure()
fig.add_trace(go.Pie(labels=labels,
                     values=values,
                     hole=.3))

fig.update_traces(textposition='inside',
                  textinfo='percent+label')

fig.show()

In [27]:
count_class2 = pd.crosstab(index = raw_dataset['categories'],
            columns = 'count')

In [28]:
count_class2

col_0,count
categories,
BBQ Joint,2
Bagel Shop,1
Bakery,1
Bistro,2
Bossam/Jokbal Restaurant,2
Buffet,2
Burger Joint,1
Café,4
Chinese Restaurant,5


In [29]:
# the ratio of categories

labels = count_class2.index
values = count_class2['count']

fig = go.Figure()
fig.add_trace(go.Pie(labels=labels,
                     values=values,
                     hole=.3))

fig.update_traces(textposition='inside',
                  textinfo='percent+label')

fig.show()

In [30]:
# looking for the names of coffee shop / cafe in Seoul
raw_dataset[(raw_dataset['categories'] == 'Coffee Shop') | (raw_dataset['categories'] == 'Café')]

,name,categories,lat,lng,id,likes,categories_classified,ranking
8,Be-Up Coffee (비읍커피),Café,37.566975,126.979706,5ba2e09d234724002ce3f415,13,Casual,3
27,Café MAMAS (카페마마스),Café,37.567216,126.979188,4b84c56bf964a520a34331e3,79,Casual,1
32,Paul Bassett (폴바셋),Coffee Shop,37.568019,126.976677,5146a5ece4b088751f847bee,139,Casual,1
45,Paul Bassett (폴바셋),Coffee Shop,37.570404,126.978832,55efabb5498ef918a753f0f2,46,Casual,2
49,Starbucks Reserve (스타벅스 리저브),Coffee Shop,37.564463,126.979025,4b5bb69af964a520a31129e3,137,Casual,1
59,CONFECTIONS BY FOUR SEASONS,Coffee Shop,37.570462,126.975299,56175cb7498e144b13689c4b,20,Casual,2
61,Starbucks (스타벅스),Coffee Shop,37.564000,126.978832,54e4032b498ea9b63904a623,15,Casual,3
64,수수커피,Café,37.571723,126.979010,59afb2389de23b1bcc2de6d8,16,Casual,3
82,Starbucks (스타벅스),Coffee Shop,37.570267,126.978990,55e24a57498e6b62f70697e0,58,Casual,1
89,Starbucks Reserve (스타벅스 리저브),Coffee Shop,37.563005,126.974359,5456b3f7498eccfb5b024079,47,Casual,2


What a Starbucks!

#### 2.4 Exploring Data in the map of Seoul

In [31]:
raw_dataset.columns

Index(['name', 'categories', 'lat', 'lng', 'id', 'likes',
       'categories_classified', 'ranking'],
      dtype='object')

In [32]:
! pip install git+https://github.com/python-visualization/branca.git@master 
# 한글 깨짐 해결을 위해 필요

  Cloning https://github.com/python-visualization/branca.git (to revision master) to /tmp/pip-req-build-a1tm8lwh
  Running command git clone -q https://github.com/python-visualization/branca.git /tmp/pip-req-build-a1tm8lwh
  Created wheel for branca: filename=branca-0.4.1+3.g5887b9b-cp36-none-any.whl size=24523 sha256=cdf9448378aee55dfc2606c00410b5cad8fc95affbe96e1f196924fb34dda81c
  Stored in directory: /tmp/pip-ephem-wheel-cache-fguyaobv/wheels/14/bc/41/d6078abe73beea93d98c014bba54c395224cabb54f95b9d248
Successfully built branca
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
  Found existing installation: branca 0.4.1
    Uninstalling branca-0.4.1:
      Successfully uninstalled branca-0.4.1


In [33]:
import folium
map_seoul = folium.Map(location=[latitude, longitude], zoom_start=15)

for lat, lng, name, categories_classified in zip(raw_dataset['lat'],
                                           raw_dataset['lng'],
                                           raw_dataset['name'],
                                           raw_dataset['categories_classified']):
  label = '{}, {}'.format(name, categories_classified)
  label = folium.Popup(label, parse_html=True)
  folium.CircleMarker(
      [lat, lng],
      radius = 4,
      popup = label,
      color = 'blue',
      fill = True,
      fill_color = '#87cefa',
      fill_opacity = 0.5,
      parse_html = False).add_to(map_seoul)

map_seoul

### 3. Methodology

In [34]:
# create dummies for linear regression modeling
reg_dataset = pd.DataFrame()

# one-hot encoding
reg_dataset = pd.get_dummies(raw_dataset['categories_classified'],
                             prefix="", prefix_sep="")

# add name, ranking, and likes columns back to dataframe
reg_dataset['ranking'] = raw_dataset['ranking']
reg_dataset['likes'] = raw_dataset['likes']
reg_dataset['name'] = raw_dataset['name']

# move name column to the first column
reg_columns = [reg_dataset.columns[-1]] + list(reg_dataset.columns[:-1])
reg_dataset = reg_dataset[reg_columns]

reg_dataset.head()

,name,American,Asian,Bar,Casual,Europe,Korean,ranking,likes
0,무교동북어국집,0,0,0,0,0,1,1,192
2,Läderach chocolatier suisse (레더라),0,0,0,1,0,0,1,57
3,Joo Ok (주옥),0,0,0,0,0,1,2,41
6,철철복집,0,0,0,0,0,1,2,46
8,Be-Up Coffee (비읍커피),0,0,0,1,0,0,3,13


### 4. Result

#### 4.1 Linear Regression

In [42]:
# multiple linear regression

msk = np.random.rand(len(reg_dataset)) < 0.8
train = reg_dataset[msk]
test = reg_dataset[~msk]

regr = linear_model.LinearRegression()
x = np.asanyarray(train[['American', 'Asian', 'Bar', 'Casual', 'Europe', 'Korean']])
y = np.asanyarray(train['likes'])
regr.fit(x, y)

# the coefficients
print('Coefficients: ', regr.coef_)
print('Intercepts: ', regr.intercept_)

Coefficients:  [-28.91574074  -4.08240741  17.33425926   9.88425926 -12.16574074
  17.94537037]
Intercepts:  54.16574074074075


In [43]:
# multiple linear regression prediction capabilities

y_hat = regr.predict(test[['American', 'Asian', 'Bar', 'Casual', 'Europe', 'Korean']])
x = np.asanyarray(test[['American', 'Asian', 'Bar', 'Casual', 'Europe', 'Korean']])
y = np.asanyarray(test['likes'])

# explained variance score: 1 is perfect prediction
print('Variance score: %.2f' % regr.score(x, y))

print('Mean absolute error: %.2f' % np.mean(np.absolute(y_hat - y)))
print('Residual sum of squares (MSE): %.2f' % np.mean((y_hat - y) ** 2))
print('R2-score: %.2f' % r2_score(y_hat, y))

Variance score: -1.01
Mean absolute error: 27.54
Residual sum of squares (MSE): 1111.07
R2-score: -2.53


#### 4.2 Logistic Regression Results

In [37]:
# Multinomial Ordinal Logistic Regression

x_train = np.asanyarray(train[['American', 'Asian', 'Bar', 'Casual', 'Europe', 'Korean']])
y_train = np.asanyarray(train['ranking'])

x_test = np.asanyarray(test[['American', 'Asian', 'Bar', 'Casual', 'Europe', 'Korean']])
y_test = np.asanyarray(test['ranking'])


# LR = LogisticRegression(C=0.01, solver='liblinear').fit(x_train, y_train)
# LR

mul_ordinal = linear_model.LogisticRegression(multi_class='multinomial',
                                              solver='newton-cg',
                                              fit_intercept=True).fit(x_train,
                                                                      y_train)

mul_ordinal

coef = mul_ordinal.coef_[0]
print (coef)

[-0.60039122  0.24072085  0.17103059  0.56726124 -0.43821348  0.05958793]


In [38]:
# Multinomial Ordinal Logistic Regression Prediction Capabilities

yhat = mul_ordinal.predict(x_test)
yhat

yhat_prob = mul_ordinal.predict_proba(x_test)
yhat_prob

jaccard_similarity_score(y_test, yhat)

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:664: FutureWarning:

jaccard_similarity_score has been deprecated and replaced with jaccard_score. It will be removed in version 0.23. This implementation has surprising behavior for binary and multiclass classification tasks.



0.4

In [39]:
log_loss(y_test, yhat_prob)

1.052560265422218

In [40]:
# Exploration of Coefficient Magnitudes of Full Dataset

x_all = np.asanyarray(reg_dataset[['American', 'Asian', 'Bar', 'Casual', 'Europe', 'Korean']])
y_all = np.asanyarray(reg_dataset['ranking'])

LR = linear_model.LogisticRegression(multi_class='multinomial',
                                            solver='newton-cg',
                                            fit_intercept=True).fit(x_all,
                                                                    y_all)

LR

coef = LR.coef_[0]
print (coef)

[-0.70999835  0.26949531  0.41067205  0.44257842 -0.48121115  0.06846177]


In [41]:
print (classification_report(y_test, yhat))

              precision    recall  f1-score   support

           1       0.40      0.40      0.40         5
           2       0.50      0.67      0.57         3
           3       0.00      0.00      0.00         2

    accuracy                           0.40        10
   macro avg       0.30      0.36      0.32        10
weighted avg       0.35      0.40      0.37        10

